In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g4cOjKX--1wpSEG20h_HfCs-l86mue52dYavnQ4OfxAmETBSLIEUwc
Mounted at /content/drive/


In [ ]:
import numpy as np

np.random.seed(12345)



In [ ]:
from tensorflow.keras.utils import to_categorical

from keras.models import Model
from keras.layers import Input, Activation, Concatenate
from keras.layers import Flatten, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D

In [ ]:
x_train = np.load('/content/drive/MyDrive/USYD/2021 S1/INFO4001/deepinsight_shuffle_img/x_train_img.npy') 
x_test = np.load('/content/drive/MyDrive/USYD/2021 S1/INFO4001/deepinsight_shuffle_img/x_test_img.npy')

# modify the default parameters of np.load



y_train = np.load('/content/drive/MyDrive/USYD/2021 S1/INFO4001/deepinsight_shuffle_img/y_train_img.npy',allow_pickle=True)
y_test = np.load('/content/drive/MyDrive/USYD/2021 S1/INFO4001/deepinsight_shuffle_img/y_test_img.npy',allow_pickle=True)


In [ ]:
label = ['16cell','2cell','4cell','8cell','blast','zygote']

n_lst=[]
for i in range(len(y_train)):
  for j in range(len(label)):
    if y_train[i] == label[j]:

      n_lst.append(j)

y_new_train = np.array(n_lst,dtype='float64')

n_lst=[]
for i in range(len(y_test)):
  for j in range(len(label)):
    if y_test[i] == label[j]:
      n_lst.append(j)

y_new_test = np.array(n_lst,dtype='float64')

In [ ]:
df_y_train = to_categorical(y_new_train,num_classes=6)

In [ ]:
x_train[0].shape

(50, 50, 3)

In [ ]:
def SqueezeNet(nb_classes, inputs=(50, 50, 3)):
    """ Keras Implementation of SqueezeNet(arXiv 1602.07360)
    @param nb_classes: total number of final categories
    Arguments:
    inputs -- shape of the input images (channel, cols, rows)
    """

    input_img = Input(shape=inputs)
    conv1 = Convolution2D(
        96, (7, 7), activation='relu', kernel_initializer='glorot_uniform',
        strides=(2, 2), padding='same', name='conv1',
        data_format="channels_last")(input_img)
    maxpool1 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool1',
        data_format="channels_last")(conv1)
    fire2_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_squeeze',
        data_format="channels_last")(maxpool1)
    fire2_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand1',
        data_format="channels_last")(fire2_squeeze)
    fire2_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand2',
        data_format="channels_last")(fire2_squeeze)
    merge2 = Concatenate(axis=1)([fire2_expand1, fire2_expand2])

    fire3_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_squeeze',
        data_format="channels_last")(merge2)
    fire3_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand1',
        data_format="channels_last")(fire3_squeeze)
    fire3_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand2',
        data_format="channels_last")(fire3_squeeze)
    merge3 = Concatenate(axis=1)([fire3_expand1, fire3_expand2])

    fire4_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_squeeze',
        data_format="channels_last")(merge3)
    fire4_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand1',
        data_format="channels_last")(fire4_squeeze)
    fire4_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand2',
        data_format="channels_last")(fire4_squeeze)
    merge4 = Concatenate(axis=1)([fire4_expand1, fire4_expand2])
    maxpool4 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool4',
        data_format="channels_last")(merge4)

    fire5_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_squeeze',
        data_format="channels_last")(maxpool4)
    fire5_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand1',
        data_format="channels_last")(fire5_squeeze)
    fire5_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand2',
        data_format="channels_last")(fire5_squeeze)
    merge5 = Concatenate(axis=1)([fire5_expand1, fire5_expand2])

    fire6_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_squeeze',
        data_format="channels_last")(merge5)
    fire6_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand1',
        data_format="channels_last")(fire6_squeeze)
    fire6_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand2',
        data_format="channels_last")(fire6_squeeze)
    merge6 = Concatenate(axis=1)([fire6_expand1, fire6_expand2])

    fire7_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_squeeze',
        data_format="channels_last")(merge6)
    fire7_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand1',
        data_format="channels_last")(fire7_squeeze)
    fire7_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand2',
        data_format="channels_last")(fire7_squeeze)
    merge7 = Concatenate(axis=1)([fire7_expand1, fire7_expand2])

    fire8_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_squeeze',
        data_format="channels_last")(merge7)
    fire8_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand1',
        data_format="channels_last")(fire8_squeeze)
    fire8_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand2',
        data_format="channels_last")(fire8_squeeze)
    merge8 = Concatenate(axis=1)([fire8_expand1, fire8_expand2])

    maxpool8 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool8',
        data_format="channels_last")(merge8)
    fire9_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_squeeze',
        data_format="channels_last")(maxpool8)
    fire9_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand1',
        data_format="channels_last")(fire9_squeeze)
    fire9_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand2',
        data_format="channels_last")(fire9_squeeze)
    merge9 = Concatenate(axis=1)([fire9_expand1, fire9_expand2])

    fire9_dropout = Dropout(0.5, name='fire9_dropout')(merge9)
    conv10 = Convolution2D(
        nb_classes, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='valid', name='conv10',
        data_format="channels_last")(fire9_dropout)

    global_avgpool10 = GlobalAveragePooling2D(data_format='channels_last')(conv10)
    softmax = Activation("softmax", name='softmax')(global_avgpool10)

    return Model(inputs=input_img, outputs=softmax)

In [ ]:
my_model = SqueezeNet(6)

In [ ]:
my_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
my_model.fit(x_train,df_y_train, batch_size=32, epochs=90, validation_split=0.2)

Epoch 1/90
6/6 [==============================] - 20s 3s/step - loss: 1.7772 - accuracy: 0.4261 - val_loss: 1.5397 - val_accuracy: 0.4884
Epoch 2/90
6/6 [==============================] - 17s 3s/step - loss: 1.7520 - accuracy: 0.4750 - val_loss: 1.6569 - val_accuracy: 0.4884
Epoch 3/90
6/6 [==============================] - 17s 3s/step - loss: 1.6684 - accuracy: 0.4533 - val_loss: 1.6749 - val_accuracy: 0.4884
Epoch 4/90
6/6 [==============================] - 17s 3s/step - loss: 1.6400 - accuracy: 0.4964 - val_loss: 1.6080 - val_accuracy: 0.4884
Epoch 5/90
6/6 [==============================] - 17s 3s/step - loss: 1.5573 - accuracy: 0.4582 - val_loss: 1.5473 - val_accuracy: 0.4884
Epoch 6/90
6/6 [==============================] - 17s 3s/step - loss: 1.4602 - accuracy: 0.4798 - val_loss: 1.5132 - val_accuracy: 0.4884
Epoch 7/90
6/6 [==============================] - 17s 3s/step - loss: 1.4800 - accuracy: 0.4570 - val_loss: 1.5032 - val_accuracy: 0.4884
Epoch 8/90
6/6 [==================

In [ ]:
df_y_test = to_categorical(y_new_test,num_classes=6)

In [ ]:
val_loss, val_acc = my_model.evaluate(x_test,df_y_test)
print(val_loss, val_acc)

2/2 [==============================] - 1s 545ms/step - loss: 0.5650 - accuracy: 0.8519
0.5649661421775818 0.8518518805503845


In [18]:
# train 5 times, record accuracy and loss. Take averages.
%%time

df_y_test = to_categorical(y_new_test,num_classes=6)

accuracy_lst=[]
loss_lst=[]

for i in range(5):
    my_model = SqueezeNet(6)
    my_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    my_model.fit(x_train,df_y_train, batch_size=32, epochs=90, validation_split=0.2)

    val_loss, val_acc = my_model.evaluate(x_test,df_y_test)

    accuracy_lst.append(val_acc)
    loss_lst.append(val_loss)



Epoch 1/90
6/6 [==============================] - 19s 3s/step - loss: 1.7890 - accuracy: 0.2834 - val_loss: 1.7714 - val_accuracy: 0.4884
Epoch 2/90
6/6 [==============================] - 17s 3s/step - loss: 1.7591 - accuracy: 0.4782 - val_loss: 1.6639 - val_accuracy: 0.4884
Epoch 3/90
6/6 [==============================] - 17s 3s/step - loss: 1.6165 - accuracy: 0.4385 - val_loss: 1.5590 - val_accuracy: 0.4884
Epoch 4/90
6/6 [==============================] - 17s 3s/step - loss: 1.5482 - accuracy: 0.4558 - val_loss: 1.5155 - val_accuracy: 0.4884
Epoch 5/90
6/6 [==============================] - 17s 3s/step - loss: 1.5113 - accuracy: 0.4565 - val_loss: 1.5342 - val_accuracy: 0.4884
Epoch 6/90
6/6 [==============================] - 17s 3s/step - loss: 1.4811 - accuracy: 0.4425 - val_loss: 1.5744 - val_accuracy: 0.4884
Epoch 7/90
6/6 [==============================] - 17s 3s/step - loss: 1.4839 - accuracy: 0.4660 - val_loss: 1.5074 - val_accuracy: 0.4884
Epoch 8/90
6/6 [==================

In [19]:
print(accuracy_lst)

[0.9074074029922485, 0.6851851940155029, 0.7777777910232544, 0.7222222089767456, 0.7222222089767456]


In [20]:
print(loss_lst)

[1.3614503145217896, 0.6040494441986084, 2.1718199253082275, 0.5465317368507385, 0.579867422580719]


In [2]:
accuracy_lst = [0.9074074029922485, 0.6851851940155029, 0.7777777910232544, 0.7222222089767456, 0.7222222089767456]
avg_cal = sum(accuracy_lst)/len(accuracy_lst)
print("The average is ", round(avg_cal,4))

The average is  0.763
